In [ ]:
!pip3 install /srv/jakenode/

Processing /srv/jakenode
  Preparing metadata (setup.py) ... done
  Created wheel for jakenode: filename=jakenode-1.0-py3-none-any.whl size=15525 sha256=ca43fcc479e6ff09ea38037a2fe41adac33b6403a22c99f3bbceb566b4aa9416
  Stored in directory: /tmp/pip-ephem-wheel-cache-bjodfzvw/wheels/83/6a/3f/1bfa175380aac4f722bcde5464b90d88357b93506792969c1b
Successfully built jakenode


In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
from datetime import datetime
from imp import reload
import os
import signal
from time import sleep

from NodeGraphQt import NodesPaletteWidget, NodesTreeWidget, PropertiesBinWidget
from Qt import QtCore, QtWidgets

from jakenode.graph_handler import GraphHandler
from jakenode.nodes.pre_built import basic_nodes, custom_ports_node, group_node, widget_nodes
from jakenode.nodes.trigger.api_trigger import APITrigger
from jakenode.nodes.outreach.sms_outreach import SMSOutreach

if __name__ == '__main__':
    init_time = datetime.now()
    print(init_time)
    print(datetime.now() - init_time)

    # handle SIGINT to make the app terminate on CTRL+C
    signal.signal(signal.SIGINT, signal.SIG_DFL)

    QtCore.QCoreApplication.setAttribute(QtCore.Qt.AA_EnableHighDpiScaling)

    app = QtWidgets.QApplication([])

    # create graph controller.
    graph = GraphHandler()

    # set up context menu for the node graph.
    graph.set_context_menu_from_file('./config/master_config.json')

    # registered example nodes.
    graph.register_nodes([
        basic_nodes.BasicNodeA,
        basic_nodes.BasicNodeB,
        custom_ports_node.CustomPortsNode,
        group_node.MyGroupNode,
        widget_nodes.DropdownMenuNode,
        widget_nodes.TextInputNode,
        widget_nodes.CheckboxNode,
        APITrigger,
        SMSOutreach
    ])

    # show the node graph widget.
    graph_widget = graph.widget
    graph_widget.resize(1100, 800)
    graph_widget.show()
    
    # Create Trigger Node
    api_trigger = graph.create_node(
        'nodes.trigger.APITrigger', text_color='#feab20')
#     trigger_node.add_label(name=trigger_node.name(), label='label<br>label2', text='text<br>line2')

    # create node with custom text color and disable it.
    n_basic_a = graph.create_node(
        'nodes.basic.BasicNodeA', text_color='#feab20')
    n_basic_a.set_disabled(True)

    # create node and set a custom icon.
    n_basic_b = graph.create_node(
        'nodes.basic.BasicNodeB', name='custom icon')
    icon = './icons/star.png'
    n_basic_b.set_icon(icon)

    # create node with the custom port shapes.
    n_custom_ports = graph.create_node(
        'nodes.custom.ports.CustomPortsNode', name='custom ports')

    # create node with the embedded QLineEdit widget.
    n_text_input = graph.create_node(
        'nodes.widget.TextInputNode', name='text node', color='#0a1e20')

    # create node with the embedded QCheckBox widgets.
    n_checkbox = graph.create_node(
        'nodes.widget.CheckboxNode', name='checkbox node')

    # create node with the QComboBox widget.
    n_combo_menu = graph.create_node(
        'nodes.widget.DropdownMenuNode', name='combobox node')

    # create group node.
    n_group = graph.create_node('nodes.group.MyGroupNode')

    # make node connections.

    # (connect nodes using the .set_output method)
    n_text_input.set_output(0, n_custom_ports.input(0))
    n_text_input.set_output(0, n_checkbox.input(0))
    n_text_input.set_output(0, n_combo_menu.input(0))
    # (connect nodes using the .set_input method)
    n_group.set_input(0, n_custom_ports.output(1))
    n_basic_b.set_input(2, n_checkbox.output(0))
    n_basic_b.set_input(2, n_combo_menu.output(1))
    # (connect nodes using the .connect_to method from the port object)
    port = n_basic_a.input(0)
    port.connect_to(n_basic_b.output(0))

    # auto layout nodes.
    graph.auto_layout_nodes()

    # crate a backdrop node and wrap it around
    # "custom port node" and "group node".
    n_backdrop = graph.create_node('Backdrop')
    n_backdrop.wrap_nodes([n_custom_ports, n_combo_menu])

    # fit nodes to the viewer.
    graph.clear_selection()
    graph.fit_to_selection()

    # Custom builtin widgets from NodeGraphQt
    # ---------------------------------------

    # create a node properties bin widget.
    properties_bin = PropertiesBinWidget(node_graph=graph)
    properties_bin.setWindowFlags(QtCore.Qt.Tool)

#     # example show the node properties bin widget when a node is double clicked.
#     def display_properties_bin(node):
#         if not properties_bin.isVisible():
#             properties_bin.show()

#     # wire function to "node_double_clicked" signal.
#     graph.node_double_clicked.connect(display_properties_bin)

    # create a nodes tree widget.
    nodes_tree = NodesTreeWidget(node_graph=graph)
    nodes_tree.set_category_label('nodeGraphQt.nodes', 'Builtin Nodes')
    nodes_tree.set_category_label('nodes.custom.ports', 'Custom Port Nodes')
    nodes_tree.set_category_label('nodes.widget', 'Widget Nodes')
    nodes_tree.set_category_label('nodes.basic', 'Basic Nodes')
    nodes_tree.set_category_label('nodes.group', 'Group Nodes')
    # nodes_tree.show()

    # create a node palette widget.
    nodes_palette = NodesPaletteWidget(node_graph=graph)
    nodes_palette.set_category_label('nodeGraphQt.nodes', 'Builtin Nodes')
    nodes_palette.set_category_label('nodes.custom.ports', 'Custom Port Nodes')
    nodes_palette.set_category_label('nodes.widget', 'Widget Nodes')
    nodes_palette.set_category_label('nodes.basic', 'Basic Nodes')
    nodes_palette.set_category_label('nodes.group', 'Group Nodes')
    # nodes_palette.show()

    app.exec_()

In [1]:
from NodeGraphQt import (
    NodeGraph,
    PropertiesBinWidget,
    NodesTreeWidget,
    NodesPaletteWidget
)

class GraphHandler(NodeGraph):

    def __init__(self, parent=None):
        self.init_time = datetime.now()
        self.display_delay_seconds = 5
        print(datetime.now() - init_time)
        super(GraphHandler, self).__init__(parent)

        # properties bin widget.
        self._prop_bin = PropertiesBinWidget(node_graph=self)
        self._prop_bin.setWindowFlags(QtCore.Qt.Tool)

        # wire signal.
        self.node_selection_changed.connect(self.handle_node_selected)
#         self.node_double_clicked.connect(self.display_prop_bin)
        self.property_changed.connect(self.handle_property_change)

    def handle_node_selected(self, node):
        if (datetime.now() - self.init_time).seconds > self.display_delay_seconds:
            for sub_node in self.all_nodes():
                if sub_node.view.isSelected():
                    self.display_node_info(sub_node)

    def handle_property_change(self, node):
        if (datetime.now() - self.init_time).seconds > self.display_delay_seconds:
            if node.view.isSelected():
                properties = node.properties()
                node_type = properties.get('type_')
                self.display_node_info(node)

    def display_node_info(self, node):
        node_types_dict = {
            'nodes.trigger.APITrigger': {'lookup_key': 'api_presets'}
        }
        node_type = node.get_property('type_')
        fetch_data_dict = node_types_dict.get(node_type)
        if fetch_data_dict:
            lookup_key = fetch_data_dict.get('lookup_key')
            print(node.get_property(lookup_key))
        

#         properties = node.properties()
#         node_type = properties.get('type_')
#         if node_type == 'nodes.trigger.TriggerAPI':
#             if node.has_property('api_summary'):
#                 node.update_label('test1')
# #                 del node.view.widgets['api_summary']
# #                 node.view.widgets['api_summary'].set_value('test')
#                 node.set_property('api_summary', 'test')#node.api_presets[0].summary)
#                 print(node.properties())
#             else:
#                 node.add_label(
#                     name='api_summary', label='Trigger Summary', text=node.api_presets[0].summary
#                 )
            
#             node.update()
#             node.update_model()

    def display_prop_bin(self, node):
        """
        function for displaying the properties bin when a node
        is double clicked
        """
        print('test')
        node.properties()
#         print(node.get_widget('my_menu').get_value())
#         if not self._prop_bin.isVisible():
#             self._prop_bin.show()

Qt.py [warning]: ImportError(QtMultimedia): libpulse-mainloop-glib.so.0: cannot open shared object file: No such file or directory


In [1]:
from datetime import datetime
init_time = datetime.now()

In [4]:
x=datetime.now() - init_time
x.seconds

18

In [2]:
from imp import reload
import os
import signal

from Qt import QtCore, QtWidgets

from NodeGraphQt import (
    NodeGraph,
    PropertiesBinWidget,
    NodesTreeWidget,
    NodesPaletteWidget
)

# import example nodes from the "example_nodes" package
from jakenode.nodes.pre_built import basic_nodes, custom_ports_node, group_node, widget_nodes
from jakenode.nodes.trigger.api import TriggerAPI

# handle SIGINT to make the app terminate on CTRL+C
signal.signal(signal.SIGINT, signal.SIG_DFL)

QtCore.QCoreApplication.setAttribute(QtCore.Qt.AA_EnableHighDpiScaling)

app = QtWidgets.QApplication([])

# create graph controller.
graph = NodeGraph()

# set up context menu for the node graph.
graph.set_context_menu_from_file('./config/master_config.json')

# registered example nodes.
graph.register_nodes([
    basic_nodes.BasicNodeA,
    basic_nodes.BasicNodeB,
    custom_ports_node.CustomPortsNode,
    group_node.MyGroupNode,
    widget_nodes.DropdownMenuNode,
    widget_nodes.TextInputNode,
    widget_nodes.CheckboxNode,
    TriggerAPI
])

# show the node graph widget.
graph_widget = graph.widget
graph_widget.resize(1100, 800)
graph_widget.show()

# Create Trigger Node
trigger_node = graph.create_node(
    'nodes.trigger.TriggerAPI', text_color='#feab20')

n_text_input = graph.create_node(
        'nodes.widget.TextInputNode', name='text node', color='#0a1e20')

n_combo_menu = graph.create_node(
        'nodes.widget.DropdownMenuNode', name='combobox node')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  """Entry point for launching an IPython kernel.
Qt.py [warning]: ImportError(QtMultimedia): libpulse-mainloop-glib.so.0: cannot open shared object file: No such file or directory
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'


In [4]:
# trigger_node.add_label(name=trigger_node.name(), label='label<br>label2', text='text<br>line2')

# print(trigger_node.properties())
# trigger_node.add_label(name='api_summary', label='Trigger Summary', text="node.api_presets[0].summary")
# # trigger_node.set_property('api_summary', 'asb')
# #.api_summary
# x=trigger_node.view
# x=x.widgets['api_summary']
trigger_node

In [3]:
trigger_node.set_property('color', (13, 18, 23, 255))

In [ ]:
from Qt import QtCore, QtWidgets
QtWidgets.QTextEdit

In [ ]:
s = 'one\none\\n'

print(s.translate(str.maketrans({'\n': '<br>'})))

In [ ]:
s.replace('\n', 'poop')

In [3]:
x = NodeGraph()

x.node_selection_changed